### Data Engineering: Optimizing Dataframes and Processing in Chunks ###

In this project we will be working to extract and load data with the proposed minimal memory usage possible( assuming that we only have 10 megabytes of memory available). In order to do so, we will utilize the capability of chunking rows of data from a dataset into memory.

The first step will involve analyzing the quality of the data and the max number of chunks or rows to process at a time. We will identify the the columns within each chunk that takes ups the most memory of the time and determine the possible number of chunks that will only utilize 5 megabytes at a time.

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 99

In [2]:
first_five = pd.read_csv('loans_2007.csv', nrows = 5)
first_five


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [3]:
first_five.keys()

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

In [4]:
print(first_five.dtypes)

id                              int64
member_id                     float64
loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                           object
int_rate                       object
installment                   float64
grade                          object
sub_grade                      object
emp_title                      object
emp_length                     object
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_d                        object
loan_status                    object
pymnt_plan                     object
purpose                        object
title                          object
zip_code                       object
addr_state                     object
dti                           float64
delinq_2yrs                   float64
earliest_cr_line               object
inq_last_6mths                float64
open_acc    

- After reviewing the following datatypes for each of the following, it was discovered that some of the columns were provided with incorrect datatypes. The following were read as the following but are suggested based on the reason also provider: 

- member_id: read as a float. However this column is for identifiable purposes and not aggregation

- id: read as an integer. This is also an identifiable column and will not be used for aggregation/ metric

- There were also columns that were metrics but were being read as a float. However at no point will the metrics become a point value. For instance, number of installments are individual installments and not partial and will always be a whole value

### Initial test of memory usage of first a thousand rows

We want to assume that our memory usage is at most 10MB. Typically, we would want to utilize no more than half of the availble memorys per chunk size -- in this case 5MB. 

Below you will see that we started with a thousand rows to have an understanding the amount of memory used and determine whether or not to decrease or increase the number of rows to process at a time. We will use the memory_usage function from Pandas and the `.sum()` to return this information is bytes. Adding in `"deep=True"` as the keyword argument to the `memory_usage` function will also examine columns that are read as objects for system level memory consumption withn the pandas dataframe.

In [5]:
first_thousand = pd.read_csv('loans_2007.csv', nrows=1000 )

In [6]:
total_mem = []
total_mem.append(first_thousand.memory_usage(deep=True).sum()/2**20)

total_mem
    

[1.5502548217773438]

### Chunk Review of first thousand rows ###

Based on our review of our first a thousand rows, we see that the resulted memory usage was around 1.55 megabytes. If we multiply 1.55MB by 3, we would have about 4.65MB of memory used per chunk which is less than half of the assumed memory available. This translate to the ability to no more than 3-thousand rows per chunk to read into memory at a time.
______

Before reading in an additional 3-thousad rows, we will need to analyze and convert conlumns that were read inaccurately. These are  columns that were being read as integers or floats and converting into objects. We first created an empty list. With that empty list, we used the memory_usage functon to read the total memory usage of each column into the list.

### Identification of column types ###

In [7]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)

strings = []
numeric= []

for chunk in chunk_iter:
    nums = chunk.select_dtypes(include=[np.number]).shape[1]
    numeric.append(nums)
    stg = chunk.select_dtypes(include=['object']).shape[1]
    strings.append(stg)

print(strings)
print(numeric)
# print the total string and numeric column break down



[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22]
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30]


### Initial review of total strings versus numerics

Above we are able to identify that the last set of chunks read a numeric value rather than an object 

In [8]:
# unique values in each string column
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)

uniques = {}
for uc in chunk_iter:
    u_values = chunk.select_dtypes(include=['object'])
    cols = u_values.columns
    for c in cols:
        val_counts = u_values[c].value_counts()
        if c in uniques:
            uniques[c].append(val_counts)
        else:
            uniques[c] = [val_counts]
            
            
uniques_combined = {}
unique_stats = {
    'column_name': [],
    'total_values': [],
    'unique_values': [],
}
for col in uniques:
    u_concat = pd.concat(uniques[col])
    u_group = u_concat.groupby(u_concat.index).sum()
    uniques_combined[col] = u_group
    if u_group.shape[0] < 50:
        print(col, u_group.shape[0])

term 1
grade 7
sub_grade 35
emp_length 11
home_ownership 4
verification_status 1
issue_d 9
loan_status 2
pymnt_plan 1
purpose 13
addr_state 42
initial_list_status 1
last_pymnt_d 45
application_type 1


In [9]:
# review columns of null values in numeric columns 

chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)

is_null = []
for uc in chunk_iter:
    flt_values = uc.select_dtypes(include=['float'])
    is_null.append(flt_values.apply(pd.isnull).sum())
    
# merge the chunk
combined_missing = pd.concat(is_null)
combined_missing.groupby(combined_missing.index).sum().sort_values()
# loop through each float colum 

member_id                        3
total_rec_int                    3
total_pymnt_inv                  3
total_pymnt                      3
revol_bal                        3
recoveries                       3
policy_code                      3
out_prncp_inv                    3
out_prncp                        3
total_rec_late_fee               3
loan_amnt                        3
last_pymnt_amnt                  3
total_rec_prncp                  3
funded_amnt_inv                  3
funded_amnt                      3
dti                              3
collection_recovery_fee          3
installment                      3
annual_inc                       7
inq_last_6mths                  32
total_acc                       32
delinq_2yrs                     32
pub_rec                         32
delinq_amnt                     32
open_acc                        32
acc_now_delinq                  32
tax_liens                      108
collections_12_mths_ex_med     148
chargeoff_within_12_

In [10]:
# Combined memory usage of compiled list 
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
series_mem= []
for chunk in chunk_iter:
    series_mem.append(chunk.memory_usage(deep=True)/ 2**20)

#combined data series together
combined_mem = pd.concat(series_mem)

# sum the total number of memory usage for entire dataset
overall_mem = combined_mem.sum()

overall_mem

66.21605968475342

In [11]:

# Are string columns consistent across chunks?
obj_cols = []
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)

for chunk in chunk_iter:
    chunk_obj_cols = chunk.select_dtypes(include=['object']).columns.tolist()
    if len(obj_cols) > 0:
        is_same = obj_cols == chunk_obj_cols
        if not is_same:
            print("overall obj cols:", obj_cols, "\n")
            print("chunk obj cols:", chunk_obj_cols, "\n")    
    else:
        obj_cols = chunk_obj_cols

overall obj cols: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 

chunk obj cols: ['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 

overall obj cols: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 



In [12]:
obj_cols

['term',
 'int_rate',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'revol_util',
 'initial_list_status',
 'last_pymnt_d',
 'last_credit_pull_d',
 'application_type']

___________

What is interesting, we see that the the `id` is not available as an object with 2 out of the 4 chunks loaded. For analsis purposes it would possibly be preferred to not have this column included as an object type. The useful object columns are listed below:

In [13]:
useful_obj_cols = ['term',
 'int_rate',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'revol_util',
 'initial_list_status',
 'last_pymnt_d',
 'last_credit_pull_d',
 'application_type']

### Determine Number of Unique Values 

We will look at each column to determine the number of unique values to determine if it would be appropriate to convert certain objects to the "Category" data type which would further reduce the total memory usage. 

In [14]:
## Create dictionary (key: column, value: list of Series objects representing each chunk's value counts)
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
str_cols_vc = {}
for chunk in chunk_iter:
    str_cols = chunk.select_dtypes(include=['object'])
    for col in str_cols.columns:
        current_col_vc = str_cols[col].value_counts()
        if col in str_cols_vc:
            str_cols_vc[col].append(current_col_vc)
        else:
            str_cols_vc[col] = [current_col_vc]

In [15]:
## Combine the value counts.
combined_vcs = {}

for col in str_cols_vc:
    combined_vc = pd.concat(str_cols_vc[col])
    final_vc = combined_vc.groupby(combined_vc.index).sum()
    combined_vcs[col] = final_vc

In [16]:
for col in useful_obj_cols:
    print(col)
    print(combined_vcs[col])
    print("-----------")

term
 36 months    31534
 60 months    11001
Name: term, dtype: int64
-----------
int_rate
  5.42%    573
  5.79%    410
  5.99%    347
  6.00%     19
  6.03%    447
          ... 
 23.59%      4
 23.91%     11
 24.11%      3
 24.40%      1
 24.59%      1
Name: int_rate, Length: 394, dtype: int64
-----------
grade
A    10183
B    12389
C     8740
D     6016
E     3394
F     1301
G      512
Name: grade, dtype: int64
-----------
sub_grade
A1    1142
A2    1520
A3    1823
A4    2905
A5    2793
B1    1882
B2    2113
B3    2997
B4    2590
B5    2807
C1    2264
C2    2157
C3    1658
C4    1370
C5    1291
D1    1053
D2    1485
D3    1322
D4    1140
D5    1016
E1     884
E2     791
E3     668
E4     552
E5     499
F1     392
F2     308
F3     236
F4     211
F5     154
G1     141
G2     107
G3      79
G4      99
G5      86
Name: sub_grade, dtype: int64
-----------
emp_title
  old palm inc                       1
 Brocade Communications              1
 CenturyLink                         1
 Depa

We were able to print the number of unique values along with the total times each values are listed with the dataset after each chunk is combined. There are list of columns that we've determined that will be eligible to convert to a category data type. Those columns were

- sub_grade
- home_ownership
- verification_status
- purpose

we are going to group these changes to into a dictionary rather than one by one as followed:

In [17]:
#will convert the following columns to category dtypes

convert_col_dtypes = {
    "sub_grade": "category", "home_ownership": "category", 
    "verification_status": "category", "purpose": "category"
}

### Data Conversions

There are some additional columns that have the ability to convert from object to either a float or integer. These columns are `term`, `int_rate` and `revol_util`. From our initial review of each columns, int_rate  and revol_util can have the '%' character removed and converted to a float data type while `term` can be stripped of the string "months" to convert to an integer

_________

We also have the opportunity to convert date strings to datetime data types. Those columns to convert are `issue_d`, `earliest_cr_line`, `last_pymnt`, and `last_credit_pull_d`

In [18]:
chunk[useful_obj_cols]

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type
42000,36 months,11.34%,C,C2,Best Buy,3 years,RENT,Not Verified,Feb-2008,Does not meet the credit policy. Status:Fully ...,n,debt_consolidation,Paying off Bills,326xx,FL,Jul-2000,100.7%,f,Feb-2011,Jun-2016,INDIVIDUAL
42001,36 months,17.66%,G,G2,CVS PHARMACY,2 years,OWN,Not Verified,Feb-2008,Does not meet the credit policy. Status:Charge...,n,debt_consolidation,School expenses from residency,787xx,TX,Mar-1989,51.9%,f,Nov-2008,Jun-2016,INDIVIDUAL
42002,36 months,15.13%,E,E4,General Motors,7 years,RENT,Not Verified,Feb-2008,Does not meet the credit policy. Status:Fully ...,n,debt_consolidation,Credit Card Consolidation,480xx,MI,Dec-1998,80.7%,f,Feb-2011,Jun-2016,INDIVIDUAL
42003,36 months,18.29%,G,G4,usa medical center,6 years,RENT,Not Verified,Feb-2008,Does not meet the credit policy. Status:Fully ...,n,debt_consolidation,want to have one bill instead of 12,366xx,AL,Jul-1995,57.2%,f,Feb-2011,Jun-2011,INDIVIDUAL
42004,36 months,10.08%,B,B3,InvestSource Inc,4 years,RENT,Not Verified,Feb-2008,Does not meet the credit policy. Status:Charge...,n,debt_consolidation,Need to pay credit card debt,928xx,CA,Sep-2005,74%,f,Mar-2010,Aug-2010,INDIVIDUAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42533,36 months,9.33%,B,B3,NaN,< 1 year,RENT,Not Verified,Jun-2007,Does not meet the credit policy. Status:Fully ...,n,other,Car repair bill,100xx,NY,NaN,NaN,f,Jun-2010,May-2007,INDIVIDUAL
42534,36 months,8.38%,A,A5,NaN,< 1 year,NONE,Not Verified,Jun-2007,Does not meet the credit policy. Status:Fully ...,n,other,Buying a car,100xx,NY,NaN,NaN,f,Jun-2010,Aug-2007,INDIVIDUAL
42535,36 months,7.75%,A,A3,Homemaker,10+ years,MORTGAGE,Not Verified,Jun-2007,Does not meet the credit policy. Status:Fully ...,n,other,Aroundthehouse,068xx,CT,NaN,NaN,f,Jun-2010,Feb-2015,INDIVIDUAL
42536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# to import columns to covert and strip percent in string to convert to numerical value

chunk_iter = pd.read_csv('loans_2007.csv', dtype=convert_col_dtypes, chunksize=3000, parse_dates=['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'])

converted = []
for chunk in chunk_iter:
    chunk['revol_util'] = chunk['revol_util'].str.strip('%').astype('float64')
    chunk['int_rate'] = chunk['int_rate'].str.strip('%').astype('float64')
    chunk['term'] = pd.to_numeric(chunk['term'].str.strip(' months'))
    converted.append(chunk.memory_usage(deep = True))


merge_con = pd.concat(converted)
final_mem = merge_con.sum()/2**20
final_mem


39.790358543395996

### Post date, float and interger data type conversions

After converting the neccessary columns to either a category or the appropriate date of numerical type, we were able to reduce the following report memory down to 39.79 Megabytes of total memory usage


In [20]:
# to collect float columns that have missing values and convert to a more efficient subtype

float_iter = pd.read_csv('loans_2007.csv', dtype=convert_col_dtypes, chunksize=3000, parse_dates=['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'])

null_cols = []
for floats in float_iter:
    floats['revol_util'] = floats['revol_util'].str.strip('%').astype('float64')
    floats['int_rate'] = floats['int_rate'].str.strip('%').astype('float64')
    floats['term'] = pd.to_numeric(floats['term'].str.strip(' months'))
    converted.append(chunk.memory_usage(deep = True))
    float_col = floats.select_dtypes(include = ['float'])
    null_cols.append(float_col.apply(pd.isnull).sum())


combined = pd.concat(null_cols)
combined.groupby(combined.index).sum().sort_values()


total_rec_prncp                  3
policy_code                      3
out_prncp_inv                    3
out_prncp                        3
total_rec_late_fee               3
member_id                        3
loan_amnt                        3
last_pymnt_amnt                  3
int_rate                         3
installment                      3
term                             3
funded_amnt_inv                  3
funded_amnt                      3
dti                              3
total_pymnt                      3
total_pymnt_inv                  3
collection_recovery_fee          3
total_rec_int                    3
revol_bal                        3
recoveries                       3
annual_inc                       7
total_acc                       32
acc_now_delinq                  32
pub_rec                         32
inq_last_6mths                  32
delinq_amnt                     32
delinq_2yrs                     32
open_acc                        32
revol_util          

In [21]:
float_iter = pd.read_csv('loans_2007.csv', dtype=convert_col_dtypes, chunksize=3000, parse_dates=['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'])

not_null_cols = []
num_rows = []
for floats in float_iter: 
    float_col = floats.select_dtypes(include = ['float'])
    not_null_cols.append(float_col.apply(pd.notna).sum())
    num_rows.append(len(floats))

combined = pd.concat(not_null_cols)
final = combined.groupby(combined.index).sum().sort_values(ascending=False)
final

total_rec_prncp               42535
out_prncp_inv                 42535
collection_recovery_fee       42535
dti                           42535
funded_amnt                   42535
funded_amnt_inv               42535
installment                   42535
last_pymnt_amnt               42535
loan_amnt                     42535
total_rec_late_fee            42535
out_prncp                     42535
member_id                     42535
policy_code                   42535
revol_bal                     42535
total_rec_int                 42535
total_pymnt_inv               42535
total_pymnt                   42535
recoveries                    42535
annual_inc                    42531
inq_last_6mths                42506
total_acc                     42506
pub_rec                       42506
delinq_amnt                   42506
delinq_2yrs                   42506
open_acc                      42506
acc_now_delinq                42506
tax_liens                     42430
collections_12_mths_ex_med  